In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib as mpl

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

ModuleNotFoundError: No module named 'geopandas'

In [3]:
files = [
    "sublist2.1.24.xlsx",
    "sublist3.1.24.xlsx",
    "sublist4.1.24.xlsx",
    "sublist5.1.24.xlsx",
    "sublist6.1.24.xlsx",
    "sublist7.1.24.xlsx",
    "sublist8.1.24.xlsx",
    "sublist9.1.24.xlsx",
    "sublist10.01.24.xlsx",
]

publication_map = {
    "MTM_PT": "Portland Press Herald / Maine Sunday Telegram", 
    "MTM_KJ": "Kennebec Journal",
    "MTM_MS":  "Morning Sentinel",
    "AMG_TR":  "Times Record",
    "SMG_SJ":  "Sun Journal",
    "SMG_AD":  "Advertisor Democrat",
    "SMG_BC":  "Bethel Citizen",
    "SMG_FJ":  "Franklin Journal",
    "SMG_LFA": "Livermore Falls Advertisor",
    "SMG_RFT": "Rumford Falls Times",
    "SMG_RH":  "Rangeley Highlander",
}

df_dict = {}

for fname in files:
    rest = fname[len("sublist"):]
    month_str = rest.split(".", 1)[0]
    month = int(month_str)
    df = pd.read_excel(fname)
    df_dict[month] = df
    df["Day pattern"] = df["Day pattern"].astype(str).str.strip()
    df["Channel"] = np.where(df["Day pattern"] == "O7Day", "Digital", "Print")
    df["State"] = df["State"].astype(str).str.strip()
    df["Zip"] = df["Zip"].astype(str).str.zfill(5)
    df["OriginalStartDate"] = pd.to_datetime(df["OriginalStartDate"], errors="coerce")
    df["LastStartDate"] = pd.to_datetime(df["LastStartDate"], errors="coerce")
    df["Pub_name"] = df["Publication"].map(publication_map)
    df["Bill Method"] = df["Bill Method"].astype(str).str.strip()
    df["Status"] = df["Status"].astype(str).str.strip()
    df["snapshot_month"] = month

In [4]:
all_df = df_dict[10]  # use only the latest month data (October 2024)

latest_active = all_df[all_df["Status"] == "Active"].copy()

snapshot_date = pd.Timestamp('2024-10-01') # assuming snapshot is at the start of October 2024

# Group publications (keep top 5, rest as "Other")
top_pubs = latest_active['Pub_name'].value_counts().head(5).index
latest_active['Pub_name_grouped'] = latest_active['Pub_name'].apply(
    lambda x: x if x in top_pubs else 'Other'
)

# tenure since first start (loyalty-ish)
latest_active["tenure_from_original_days"] = (
    snapshot_date - latest_active["OriginalStartDate"]
).dt.days

# age of current term (how long the current start has been running)
latest_active["age_of_current_term_days"] = (
    snapshot_date - latest_active["LastStartDate"]
).dt.days

latest_active["renewal_gap_days"] = (
    latest_active["LastStartDate"] - latest_active["OriginalStartDate"]
).dt.days


conditions = [
    latest_active["Bill Method"].str.contains("Auto Pay - CC", case=False, na=False),
    latest_active["Bill Method"].str.contains("Office Pay", case=False, na=False)
]
choices = ["Auto Pay - CC", "Office Pay"]
latest_active["BillMethod_group"] = np.select(conditions, choices, default="Other")

# Remove rows with missing critical data
df_clean = latest_active.dropna(subset=[
    'tenure_from_original_days', 
    'age_of_current_term_days', 
    'renewal_gap_days',
]).copy()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

numeric_features = [
    "tenure_from_original_days",   # older vs newer
    "age_of_current_term_days",    # how recent the current start is
    "renewal_gap_days", # 0 = continuous, >0 = had a break
]

categorical_features = [
    "Channel",           # Digital vs Print
    "BillMethod_group",  # AutoPay vs Other
    "Pub_name",          # which publication
]

#one-hot encode categoricals
X_cat = pd.get_dummies(df_clean[categorical_features], drop_first=False)

# scale numeric features
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(df_clean[numeric_features])

# combine into final matrix
X = np.hstack([X_num_scaled, X_cat.values])

k = 3  # you can try 3, 4, 5 and compare
kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
df_clean["cluster"] = kmeans.fit_predict(X)

print("Numeric feature means by cluster:")
print(df_clean.groupby("cluster")[numeric_features].mean())

print("\nChannel distribution by cluster:")
print(
    df_clean.groupby("cluster")["Channel"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
)

print("\nBill method distribution by cluster:")
print(
    df_clean.groupby("cluster")["BillMethod_group"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
)

print("\nPublication distribution by cluster (top 5 per cluster):")
for c in sorted(df_clean["cluster"].unique()):
    print(f"\nCluster {c}:")
    print(df_clean[df_clean["cluster"] == c]["Pub_name"]
          .value_counts(normalize=True)
          .head(5))




In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# assuming you already have X and df_clean["cluster"]
pca = PCA(n_components=2)
X_2d = pca.fit_transform(X)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    X_2d[:, 0],
    X_2d[:, 1],
    c=df_clean["cluster"],
    alpha=0.6
)
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.title("Subscribers in PCA space colored by cluster")
plt.colorbar(scatter, label="Cluster")
plt.show()


In [ ]:
cluster_means = df_clean.groupby("cluster")[numeric_features].mean()

cluster_means.plot(kind="bar", figsize=(8, 5))
plt.title("Numeric feature means by cluster")
plt.ylabel("Mean (original scale)")
plt.xticks(rotation=0)
plt.show()


In [ ]:
channel_dist = (
    df_clean.groupby("cluster")["Channel"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
)

channel_dist.plot(kind="bar", stacked=True, figsize=(8, 5))
plt.title("Channel distribution by cluster")
plt.ylabel("Proportion")
plt.xticks(rotation=0)
plt.show()


In [ ]:
channel_dist = (
    df_clean.groupby("cluster")["Pub_name"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
)

channel_dist.plot(kind="bar", stacked=True, figsize=(8, 5))
plt.title("Publications distribution by cluster")
plt.ylabel("Proportion")
plt.xticks(rotation=0)
plt.show()


In [ ]:
channel_dist = (
    df_clean.groupby("cluster")["BillMethod_group"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
)

channel_dist.plot(kind="bar", stacked=True, figsize=(8, 5))
plt.title("Bill Methods distribution by cluster")
plt.ylabel("Proportion")
plt.xticks(rotation=0)
plt.show()


In [ ]:
# --------this page plots a double bar chart to show the trend of Digital vs print for each month of all locations

channel_dict = {}
for month, df in df_dict.items():
    digital_count = (df["Channel"] == "Digital").sum()
    print_count   = (df["Channel"] == "Print").sum()
    channel_dict[month] = {'Print': print_count, 'Digital': digital_count}

months = sorted(channel_dict.keys())
print_counts   = np.array([channel_dict[m]['Print'] for m in months])
digital_counts = np.array([channel_dict[m]['Digital'] for m in months])
x = np.arange(len(months))

# --- month-over-month growth rates (in %) ---
print_growth   = np.diff(print_counts) / print_counts[:-1] * 100
digital_growth = np.diff(digital_counts) / digital_counts[:-1] * 100
x_growth = x[1:]  # growth is between months, so starts at second month

fig, ax1 = plt.subplots(figsize=(10, 6))

# --- stacked bars: Print at bottom, Digital on top ---
bar_width = 0.6  # a bit wider since we only have one stack per month

bars_print = ax1.bar(
    x,
    print_counts,
    width=bar_width,
    label='Print'
)

bars_digital = ax1.bar(
    x,
    digital_counts,
    width=bar_width,
    bottom=print_counts,  # stack Digital on top of Print
    label='Digital'
)

ax1.set_xlabel('Month')
ax1.set_ylabel('Number of Subscribers')
ax1.set_xticks(x)
ax1.set_xticklabels(months)

# --- second axis: growth rates ---
ax2 = ax1.twinx()
line_print_growth, = ax2.plot(
    x_growth,
    print_growth,
    marker='o',
    linestyle='--',
    color='red',
    label='Print MoM %'
)
line_digital_growth, = ax2.plot(
    x_growth,
    digital_growth,
    marker='o',
    linestyle='-',
    color='green',
    label='Digital MoM %'
)
ax2.set_ylabel('MoM Growth (%)')

# --- combined legend (bars + lines) ---
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(
    handles1 + handles2,
    labels1 + labels2,
    loc='upper left'
)

plt.title('Print vs Digital Subscribers and MoM Growth')
plt.tight_layout()
plt.show()


In [ ]:
all_pub_series = pd.concat([df["Pub_name"] for df in df_dict.values()])
global_top3 = all_pub_series.value_counts().head(3).index.tolist()

months = sorted(df_dict.keys())
cols = global_top3 + ["Other"]

summary = pd.DataFrame(0, index=months, columns=cols, dtype=int)

for month, df_m in df_dict.items():
    counts = df_m["Pub_name"].value_counts()
    for pub in global_top3:
        summary.loc[month, pub] = counts.get(pub, 0)
    other_count = counts[~counts.index.isin(global_top3)].sum()
    summary.loc[month, "Other"] = other_count

# 2) MoM growth (%)
growth = summary[global_top3].pct_change() * 100   # first row NaN

# 3) Plot: stacked bars + MoM lines
fig, ax1 = plt.subplots(figsize=(10, 6))

bar_colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "lightgray"]  # top3 + Other
summary.plot(
    kind="bar",
    stacked=True,
    ax=ax1,
    color=bar_colors
)

ax1.set_xlabel("Month")
ax1.set_ylabel("Count")
ax1.set_title("Top 3 Publications vs Others by MoM Growth")

x = np.arange(len(months))

# Choose distinct colors for the MoM lines
line_colors = ["red", "black", "magenta"]  # one per top3 pub

ax2 = ax1.twinx()
for i, pub in enumerate(global_top3):
    ax2.plot(
        x,
        growth[pub].values,     # NaN at first month is fine
        marker='o',
        linestyle='--',
        color=line_colors[i],
        label=f"{pub} MoM %",
    )

ax2.set_ylabel("MoM Growth (%)")

# Combine legends
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(
    handles1 + handles2,
    labels1 + labels2,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    title="Publication"
)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 1️⃣ Combine all months into one big DataFrame
all_df = pd.concat(df_dict.values(), ignore_index=True)

# -------------------------------
#  First chart:
#  Top Maine Cities by Subscriber Count (Digital vs Print)
# -------------------------------

TOP_N = 10  

# Filter to just Maine
maine_df = all_df[all_df["State"] == "ME"]

# Group by City + Channel, count subscribers (rows)
city_channel_counts = (
    maine_df
    .groupby(["City", "Channel"])
    .size()
    .unstack(fill_value=0)   # columns: Print, Digital
)

# Add total subscribers to sort on
city_channel_counts["Total"] = city_channel_counts.sum(axis=1)

# Take top N cities
top_cities = city_channel_counts.sort_values("Total", ascending=False).head(TOP_N)

# Plot Digital vs Print for top Maine cities
ax = top_cities[["Print", "Digital"]].plot(
    kind="bar",
    figsize=(12, 6)
)

plt.title("Top Maine Cities by Subscriber Count (Digital vs Print)")
plt.xlabel("City")
plt.ylabel("Subscriber Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# -------------------------------
#  Second chart:
#  Top States Outside Maine by Subscriber Count (Digital vs Print)
# -------------------------------

# Filter to states that are NOT Maine
outside_me_df = all_df[all_df["State"] != "ME"]

state_channel_counts = (
    outside_me_df
    .groupby(["State", "Channel"])
    .size()
    .unstack(fill_value=0)
)

state_channel_counts["Total"] = state_channel_counts.sum(axis=1)

top_states = state_channel_counts.sort_values("Total", ascending=False).head(TOP_N)

ax2 = top_states[["Print", "Digital"]].plot(
    kind="bar",
    figsize=(12, 6)
)

plt.title("Top States Outside Maine by Subscriber Count (Digital vs Print)")
plt.xlabel("State")
plt.ylabel("Subscriber Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Make sure Bill Method is cleaned
all_df["Bill Method"] = all_df["Bill Method"].astype(str).str.strip()
all_df["Bill Method"] = all_df["Bill Method"].replace("", np.nan)

# Split Maine vs Outside Maine
maine_df      = all_df[all_df["State"] == "ME"]
outside_me_df = all_df[all_df["State"] != "ME"]

TOP_N = 8  # how many bill methods to show (by total volume)

# -------------------------------
# 1️⃣ Inside Maine: Digital vs Print by Bill Method
# -------------------------------
bm_ch_me = (
    maine_df
    .groupby(["Bill Method", "Channel"])
    .size()
    .unstack(fill_value=0)          # columns: Print, Digital
)

# keep only top N bill methods by total count
bm_ch_me["Total"] = bm_ch_me.sum(axis=1)
bm_ch_me_top = bm_ch_me.sort_values("Total", ascending=False).head(TOP_N)
bm_ch_me_top = bm_ch_me_top[["Print", "Digital"]]  # keep only these columns

ax = bm_ch_me_top.plot(
    kind="bar",
    figsize=(12, 6)
)

plt.title("Bill Method by Channel Inside Maine")
plt.xlabel("Bill Method")
plt.ylabel("Subscriber Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# -------------------------------
# 2️⃣ Outside Maine: Digital vs Print by Bill Method
# -------------------------------
bm_ch_out = (
    outside_me_df
    .groupby(["Bill Method", "Channel"])
    .size()
    .unstack(fill_value=0)
)

bm_ch_out["Total"] = bm_ch_out.sum(axis=1)
bm_ch_out_top = bm_ch_out.sort_values("Total", ascending=False).head(TOP_N)
bm_ch_out_top = bm_ch_out_top[["Print", "Digital"]]

ax2 = bm_ch_out_top.plot(
    kind="bar",
    figsize=(12, 6)
)

plt.title("Bill Method by Channel Outside Maine")
plt.xlabel("Bill Method")
plt.ylabel("Subscriber Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [21]:
# ---------- heat map of top publication, digital increase rate, subscriber increase rate, and total subscriber of each county
zip_dict = {}

for month, df in df_dict.items():
    zip_num = pd.to_numeric(df["Zip"], errors="coerce")
    mask = (df["State"] == "ME") & zip_num.between(3901, 4992)
    me_zip = df.loc[mask].copy()
    zip_counts_all = me_zip.groupby("Zip").size()
    zip_counts_digital = (
        me_zip[me_zip["Channel"] == "Digital"]
        .groupby("Zip")
        .size()
    )
    zip_dict[month] = {
        "total": zip_counts_all.to_dict(),
        "digital": zip_counts_digital.to_dict(),
    }

month_start = 2
month_end = 10

total_2   = zip_dict[month_start]["total"]
total_10  = zip_dict[month_end]["total"]
digital_2  = zip_dict[month_start]["digital"]
digital_10 = zip_dict[month_end]["digital"]

# all zip codes that appear in either month
all_zips = set(total_2.keys()) | set(total_10.keys()) | set(digital_2.keys()) | set(digital_10.keys())

total_change = {}
digital_change = {}

for z in all_zips:
    total_change[z] = total_10.get(z, 0) - total_2.get(z, 0)
    digital_change[z] = digital_10.get(z, 0) - digital_2.get(z, 0)



In [24]:
zcta = gpd.read_file("tl_2020_us_zcta520/tl_2020_us_zcta520.shp")
zcta["Zip_num"] = zcta["ZCTA5CE20"].astype(int)
zcta_me = zcta[zcta["Zip_num"].between(3901, 4992)].copy()

# Make a clean Zip column as 5-digit string
zcta_me["Zip"] = zcta_me["ZCTA5CE20"].astype(str).str.zfill(5)

# 3. Build change_df from your dicts
change_df = pd.DataFrame({
    "Zip": list(total_change.keys()),
    "total_change": list(total_change.values()),
})
change_df["digital_change"] = change_df["Zip"].map(digital_change)

# ensure Zip is also 5-digit string
change_df["Zip"] = change_df["Zip"].astype(int).astype(str).str.zfill(5)

# 4. Join change data to *Maine-only* ZCTAs
ma_change = zcta_me.merge(change_df, on="Zip", how="left")

# Fill missing values with 0 (no change data for that ZIP)
ma_change[["total_change", "digital_change"]] = (
    ma_change[["total_change", "digital_change"]].fillna(0)
)

In [ ]:
# 5. Plot a heat map (example: total_change)
fig, ax = plt.subplots(figsize=(8, 10))
change = [item for key, item in total_change.items()]

cmap = mpl.cm.viridis
bounds = [-10, 0, 10, 20, 30, 50, 80]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')

ma_change.plot(
    column="total_change",
    ax=ax,
    cmap=cmap,
    norm=norm,                 # <- tell it to use your BoundaryNorm
    edgecolor="black",
    linewidth=0.2,
    legend=True,
    legend_kwds={              # <- make the colorbar match your bins
        "boundaries": bounds,
        "ticks": bounds,
        "spacing": "proportional",
        "label": "Total change",
        "extend": "both",      # to show arrows above/below range
    },
)

ax.set_title("Change in Total Subscribers by ZIP (Feb - Oct 2024)")
ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 10))

cmap = mpl.cm.viridis
bounds = [-10, 0, 10, 20, 30, 50, 80]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')

ma_change.plot(
    column="digital_change",
    ax=ax,
    cmap=cmap,
    norm=norm,                 # <- tell it to use your BoundaryNorm
    edgecolor="black",
    linewidth=0.2,
    legend=True,
    legend_kwds={              # <- make the colorbar match your bins
        "boundaries": bounds,
        "ticks": bounds,
        "spacing": "proportional",
        "label": "Digital change",
        "extend": "both",      # to show arrows above/below range
    },
)

ax.set_title("Change in Digital Subscription by ZIP (Feb - Oct 2024)")
ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
df_subscribers = pd.DataFrame(
    list(zip_dict[10]['total'].items()),
    columns=["Zip", "count"]
)
df_subscribers["Zip"] = df_subscribers["Zip"].astype(int).astype(str).str.zfill(5)
ma_subscribers = zcta_me.merge(df_subscribers, on="Zip", how="left")
ma_subscribers["count"] = ma_subscribers["count"].fillna(0)
fig, ax = plt.subplots(figsize=(8, 10))


cmap = mpl.cm.viridis
bounds = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')


ma_subscribers.plot(
    column="count",      # this is the value you just merged in
    ax=ax,
    cmap=cmap,
    norm=norm,                 # <- tell it to use your BoundaryNorm
    edgecolor="black",
    linewidth=0.2,
    legend=True,
    legend_kwds={              # <- make the colorbar match your bins
        "boundaries": bounds,
        "ticks": bounds,
        "spacing": "proportional",
        "label": "Total subscribers",
        "extend": "both",
    },
)

ax.set_title("Total Subscribers by ZIP (Oct 2024)")  # or "Feb–Oct 2024" if that’s what you mean
ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# -------------------------
# 1. Build a GDF per month
# -------------------------
months = sorted(zip_dict.keys())   # e.g. [2,3,4,5,6,7,8,9,10]

ma_subscribers_by_month = {}

for m in months:
    df_subscribers = pd.DataFrame(
        list(zip_dict[m]['total'].items()),
        columns=["Zip", "count"]
    )
    df_subscribers["Zip"] = (
        df_subscribers["Zip"].astype(int).astype(str).str.zfill(5)
    )
    
    gdf_m = zcta_me.merge(df_subscribers, on="Zip", how="left")
    gdf_m["count"] = gdf_m["count"].fillna(0)
    ma_subscribers_by_month[m] = gdf_m

# -------------------------
# 2. Global min/max for color scale
# -------------------------
all_counts = pd.concat(
    [gdf["count"] for gdf in ma_subscribers_by_month.values()],
    axis=0
)
vmin = all_counts.min()
vmax = all_counts.max()

# -------------------------
# 3. Set up figure, colormap, and ONE colorbar
# -------------------------
cmap = mpl.cm.viridis

fig, ax = plt.subplots(figsize=(8, 10))

norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Total subscribers")

month_labels = {
    2: "Feb 2024",
    3: "Mar 2024",
    4: "Apr 2024",
    5: "May 2024",
    6: "Jun 2024",
    7: "Jul 2024",
    8: "Aug 2024",
    9: "Sep 2024",
    10: "Oct 2024",
}

# -------------------------
# 4. Animation function
# -------------------------
def update(frame):
    ax.clear()  # clears map but not colorbar

    month = months[frame]
    gdf = ma_subscribers_by_month[month]

    gdf.plot(
        column="count",
        ax=ax,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        edgecolor="black",
        linewidth=0.2,
        legend=False,   # <- important: no new colorbar each frame
    )

    label = month_labels.get(month, f"Month {month}")
    ax.set_title(f"Total Subscribers by ZIP ({label})")
    ax.axis("off")

# -------------------------
# 5. Create and display animation in Jupyter
# -------------------------
anim = FuncAnimation(
    fig,
    update,
    frames=len(months),
    interval=1000,
    repeat=True
)

HTML(anim.to_jshtml())